<a href="https://colab.research.google.com/github/aashishpiitk/from-Scratch/blob/master/Unet_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import pandas as pd
import os, math, sys
import glob, itertools
import argparse, random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.models import vgg19
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import save_image, make_grid

import plotly
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

random.seed(42)
import warnings
warnings.filterwarnings("ignore")

import math
from torchsummary import summary
import torchvision
from PIL import Image, ImageOps

In [ ]:
class Unet_downsample_block(nn.Module):
  def __init__(self, in_channels, out_channels, num_convs=2, isLast=False):
    super(Unet_downsample_block, self).__init__()

    self.num_convs = num_convs
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.isLast = isLast

    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding=0, stride=1)
    self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding=0, stride=1)
    self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)## to decrease the resolution to half
    self.relu = nn.ReLU()

  def forward(self, x):
    for i in range(self.num_convs):
      if(i==0):
        x = self.conv1(x)
      else:
        x = self.conv2(x)
      x = self.relu(x)
    if(not self.isLast):
      x = self.maxpool(x)
    return x

In [ ]:
class Unet_upsample_block(nn.Module):
  def __init__(self, in_channels, out_channels, num_convs=2, isLast=False):
    super(Unet_upsample_block, self).__init__()

    self.num_convs = num_convs
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.isLast = isLast

    self.conv1 = nn.Conv2d(in_channels, 2*out_channels, kernel_size = 3, padding=0, stride=1)
    self.conv2 = nn.Conv2d(2*out_channels, 2*out_channels, kernel_size = 3, padding=0, stride=1)
    self.up_conv = nn.ConvTranspose2d(2*out_channels, out_channels, kernel_size=2, stride=2)
    #self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)## to decrease the resolution to half
    self.relu = nn.ReLU()

  

  def crop(self, x, encoder_features):
    _, _, H, W = x.shape
    encoder_features = torchvision.transforms.CenterCrop((H, W))(encoder_features)
    return encoder_features

  
  def forward(self, x, encoder_features):
    #print(encoder_features.shape)
    x = torch.cat((x, self.crop(x, encoder_features)), dim=1)
    #print(x.shape, encoder_features.shape)

    for i in range(self.num_convs):
      if(i==0):
        x = self.conv1(x)
      else:
        x = self.conv2(x)
      x = self.relu(x)

    if(not self.isLast):
      x = self.up_conv(x) 
    return x

  

In [ ]:
# conv_arch = [(), ]

In [ ]:
class UNet(nn.Module):
  def __init__(self, in_channels):
    super(UNet, self).__init__()

    self.d1 = Unet_downsample_block(in_channels, 64)
    self.d2 = Unet_downsample_block(64, 128)
    self.d3 = Unet_downsample_block(128, 256)
    self.d4 = Unet_downsample_block(256, 512)
    self.d5 = Unet_downsample_block(512, 1024, isLast=True)

    self.u0 = nn.ConvTranspose2d(1024,512,2,2)
    self.u1 = Unet_upsample_block(1024, 256)
    self.u2 = Unet_upsample_block(512, 128)
    self.u3 = Unet_upsample_block(256, 64)
    self.u4 = Unet_upsample_block(128, 32, isLast=True)
    self.u5 = nn.Conv2d(64, 64, 1, 1)
     

  def forward(self, x):
    x1 = self.d1(x)
    x2 = self.d2(x1)
    x3 = self.d3(x2)
    x4 = self.d4(x3)
    x5 = self.d5(x4)
    #print(x1.shape, x2.shape, x3.shape, x4.shape, x5.shape)
    y1 = self.u0(x5)
    y2 = self.u1(y1, x4)
    y3 = self.u2(y2, x3)
    y4 = self.u3(y3, x2)
    y5 = self.u4(y4, x1)
    output = self.u5(y5)

    return output




In [ ]:
unet = UNet(3)

In [ ]:
summary(unet, (3,572,572))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 570, 570]           1,792
              ReLU-2         [-1, 64, 570, 570]               0
            Conv2d-3         [-1, 64, 568, 568]          36,928
              ReLU-4         [-1, 64, 568, 568]               0
         MaxPool2d-5         [-1, 64, 284, 284]               0
Unet_downsample_block-6         [-1, 64, 284, 284]               0
            Conv2d-7        [-1, 128, 282, 282]          73,856
              ReLU-8        [-1, 128, 282, 282]               0
            Conv2d-9        [-1, 128, 280, 280]         147,584
             ReLU-10        [-1, 128, 280, 280]               0
        MaxPool2d-11        [-1, 128, 140, 140]               0
Unet_downsample_block-12        [-1, 128, 140, 140]               0
           Conv2d-13        [-1, 256, 138, 138]         295,168
             ReLU-14        [-1,

In [ ]:
## Model is ready now we need to prepare the dataset and write the training loop


In [ ]:
### PREPARATION OF DATA AND UNDERSTANDING THE DATA
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aashishpatel","key":"d1f9de0ad4f74c0df5d828f9e6a73ee5"}'}

In [ ]:
% mkdir ~/.kaggle/

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 /root/.kaggle/kaggle.json

In [ ]:
! mkdir siim-acr && cd siim-acr 
! mkdir data && cd data

In [ ]:
pwd

'/content'

In [ ]:
% cd siim-acr

/content/siim-acr


In [ ]:
% mkdir data

In [ ]:
% cd data

/content/siim-acr/data


In [ ]:
! kaggle datasets download -d seesee/siim-train-test --force

 99% 1.90G/1.92G [00:14<00:00, 163MB/s]
100% 1.92G/1.92G [00:14<00:00, 146MB/s]


In [ ]:
! unzip siim-train-test.zip 
! mv siim/* . 
! rmdir siim
! mkdir ../src/ && cd ../src
! git clone https://github.com/sneddy/pneumothorax-segmentation

Streaming output truncated to the last 5000 lines.
  inflating: siim/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.31989.1517875157.877995/1.2.276.0.7230010.3.1.3.8323329.31989.1517875157.877994/1.2.276.0.7230010.3.1.4.8323329.31989.1517875157.877996.dcm  
  inflating: siim/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.3199.1517875176.555630/1.2.276.0.7230010.3.1.3.8323329.3199.1517875176.555629/1.2.276.0.7230010.3.1.4.8323329.3199.1517875176.555631.dcm  
  inflating: siim/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.31990.1517875157.880306/1.2.276.0.7230010.3.1.3.8323329.31990.1517875157.880305/1.2.276.0.7230010.3.1.4.8323329.31990.1517875157.880307.dcm  
  inflating: siim/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.31991.1517875157.897107/1.2.276.0.7230010.3.1.3.8323329.31991.1517875157.897106/1.2.276.0.7230010.3.1.4.8323329.31991.1517875157.897108.dcm  
  inflating: siim/dicom-images-train/1.2.276.0.7230010.3.1.2.8323329.31992.1517875157.897980/1.2.276.0.72300

In [ ]:
! rm -r /content/train-rle.csv

In [ ]:
! pip install pydicom

     |████████████████████████████████| 1.9MB 13.5MB/s 


In [ ]:
! export OUTPUT_DIR='../data/dataset512'

In [ ]:
! python pneumothorax-segmentation/unet_pipeline/utils/prepare_png.py -train_path ../data/dicom-images-train/ -test_path ../data/dicom-images-test/ -out_path $OUTPUT_DIR -img_size 512 -rle_path ../data/train-rle.csv

Traceback (most recent call last):
  File "pneumothorax-segmentation/unet_pipeline/utils/prepare_png.py", line 98, in <module>
    main()
  File "pneumothorax-segmentation/unet_pipeline/utils/prepare_png.py", line 94, in main
    save_train(train_fns, rle, out_path, img_size, n_train, n_threads)
  File "pneumothorax-segmentation/unet_pipeline/utils/prepare_png.py", line 62, in save_train
    if os.path.isdir(out_path):
  File "/usr/lib/python3.7/genericpath.py", line 42, in isdir
    st = os.stat(s)
TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType
